Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [oracleselixir.com](https://oracleselixir.com/tools/downloads).
<hr>

# League of Legends Regions Picks + Bans

This Notebook contains code releated to the data cleaning and calculations for my [Tableau Dashboard](). This looks at how often each champion was picked, banned, how often that champion won and what role it was played in the most. As every region plays a different number of games, the position of each champion is normalized as pick+banned per 100 games (the `normalize` function below) which takes picked count, banned count and total games played for each region and returns a series of picked/banned rate.
<hr>

In [1]:
import pandas as pd

def normalize(row):
    x, y, games = row

    x = (x/games) * 100
    y = (y/games) * 100

    return pd.Series([x, y])

In [2]:
# load data ( from oracle elixer )
df = pd.read_csv('./data/raw/2021.csv', parse_dates=['date'])

# wanted regions
df = df[df['league'].isin(['LCS', 'LEC', 'LPL', 'LCK', 'CBLOL', 'LCO', 'LJL'])]

# get total games by regions
region_games = df.groupby('league')['gameid'].nunique().to_frame()
region_games = region_games.rename(columns={'gameid': 'games_count'}).reset_index()

# remove no champions ( i think these are team totals? )
df = df[~df['champion'].isnull()]

# get most common role for each champion
role = df.groupby(['champion', 'position']).size().reset_index()
role = role.sort_values(by=0, ascending=False).drop(columns=[0])
role.to_csv('./data/files/champ_role.csv', index=False)

In [3]:
# get total picked by region
picked = df.groupby(['league', 'champion']).size().reset_index()
picked = picked.rename(columns={0: 'picked_count'}).merge(region_games)
picked['pick_percent'] = picked['picked_count']/picked['games_count']

# get winrate by champion
win = df.groupby(['league', 'champion'])['result'].sum().reset_index()
win = win.rename(columns={'result': 'win_count'}).merge(picked[['league', 'champion', 'picked_count']])
win['win_percent'] = win['win_count']/win['picked_count']

In [4]:
# get total bans by region
cols = ['gameid', 'side', 'league', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5']
bans = df[cols].copy().drop_duplicates()

bans = bans.melt(id_vars=['gameid', 'side', 'league']).rename(columns={'value': 'champion'})
bans = bans.drop(columns=['side', 'variable'])
bans = bans[~bans['champion'].isnull()].drop_duplicates()

total_bans = bans.groupby(['league', 'champion']).size().reset_index()
total_bans = total_bans.rename(columns={0: 'ban_count'}).merge(region_games)
total_bans['ban_percent'] = total_bans['ban_count']/total_bans['games_count']
total_bans = total_bans.drop(columns=['games_count'])

In [5]:
# merge picks and bans together
pick_ban = picked.merge(total_bans, on=['league', 'champion'], how='outer').merge(region_games).fillna(0)

pick_ban['pick_or_ban'] = pick_ban['picked_count'] + pick_ban['ban_count']
pick_ban['pick_ban_pct'] = pick_ban['pick_or_ban']/pick_ban['games_count']

# noramlize by games played
pick_ban[['pick_rate', 'ban_rate']] = pick_ban[['picked_count', 'ban_count', 'games_count']].apply(normalize, axis=1)

# add winrate to picks + bans
pick_ban = pick_ban.merge(win)

# save file
pick_ban.to_csv('./data/clean/pick_bans.csv', index=False)